<a href="https://colab.research.google.com/github/Nguyen-Kim-Son/large-language-models/blob/main/DeepSeek_R1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cài đặt thư viện
!pip install transformers datasets unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.

# Mô tả bộ dữ liệu lập luận tổng hợp: "A dataset consisting of text instructions for generating Python code to perform geospatial data analysis and mapping."

In [3]:
!pip install transformers peft



In [4]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
MODEL = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"

In [6]:
from huggingface_hub import notebook_login

notebook_login()


In [7]:
from datasets import load_dataset

# Tải dataset
dataset = load_dataset("nguyenkimson0202/my-distiset-7cc3ecde", split="train")

# In ra các cột
print(dataset.column_names)
# In ra vài hàng đầu tiên để kiểm tra
print(dataset[:5])


README.md:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/7.25k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20 [00:00<?, ? examples/s]

['text', 'label']
{'text': ['Create a Python script that utilizes the geopandas library to merge two shapefiles (.shp) based on a common field, then calculate the area of each polygon using the area attribute of geopandas and plot the results using matplotlib. Assume the shapefiles are located in the same directory as the script.', 'Create a bar chart using matplotlib to visualize the count of cadastral parcels per county in a GeoDataFrame containing property boundaries and county information. The x-axis should represent county names and the y-axis should display the count of parcels. Use a suitable colormap to differentiate between counties with a high number of parcels and those with a low number. Use a legend to distinguish between different color levels in the bar chart.', 'Develop a Python script that utilizes geopandas to create a choropleth map displaying the population density of different administrative regions in a given country, using a dataset containing the region boundari

In [8]:
!pip install bitsandbytes


In [20]:
#Fine-tune mô hình DeepSeek
#Tải mô hình và tokenizer:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from peft import get_peft_model, LoraConfig

model = AutoModelForCausalLM.from_pretrained("unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit")
tokenizer = AutoTokenizer.from_pretrained("unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit")

# Định nghĩa prompt style
prompt_style = """Below is an instruction that describes a task...
[your prompt template here]
"""

# Tải dataset
dataset = load_dataset("nguyenkimson0202/my-distiset-7cc3ecde", split="train")

# Định nghĩa hàm formatting
def formatting_prompts_func(examples):
    texts = examples["text"]
    formatted_texts = []
    for text in texts:
        formatted_text = prompt_style.format(text, "") + tokenizer.eos_token
        formatted_texts.append(formatted_text)
    return {
        "text": formatted_texts,
    }

# Áp dụng formatting
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)
def tokenize_function(examples):
    outputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    # Sử dụng `input_ids` như `labels` để mô hình dự đoán từ tiếp theo
    outputs["labels"] = outputs["input_ids"].clone()
    return outputs



tokenized_datasets = formatted_dataset.map(tokenize_function, batched=True)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # False nếu không sử dụng Masked Language Modeling
)

# Cấu hình LoRA
lora_config = LoraConfig(
    r=16,
    bias="none"
)

# Thêm LoRA vào mô hình
model = get_peft_model(model, lora_config)

# Thiết lập tham số huấn luyện với tối ưu bộ nhớ
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    fp16=True
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)
# Huấn luyện mô hình
trainer.train()



`low_cpu_mem_usage` was None, now default to True since model is quantized.


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=6, training_loss=6.009981791178386, metrics={'train_runtime': 15.5322, 'train_samples_per_second': 3.863, 'train_steps_per_second': 0.386, 'total_flos': 189959378042880.0, 'train_loss': 6.009981791178386, 'epoch': 2.0})

In [25]:
# Đánh giá mô hình
# Tải tập dữ liệu kiểm tra
from datasets import load_dataset, DatasetDict

# Tải dataset
dataset = load_dataset("nguyenkimson0202/my-distiset-7cc3ecde", split="train")

# Chia dataset thành train và validation
split_dataset = dataset.train_test_split(test_size=0.2)

# Lưu lại các phần tách ra
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

# Mã hóa dữ liệu huấn luyện
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)

# Mã hóa dữ liệu kiểm tra
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator
)

# Huấn luyện mô hình
trainer.train()

# Đánh giá mô hình
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Step,Training Loss


Evaluation results: {'eval_runtime': 0.2735, 'eval_samples_per_second': 14.625, 'eval_steps_per_second': 3.656, 'epoch': 5.0}


In [26]:
model.save_pretrained('./final_model')
tokenizer.save_pretrained('./final_model')


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/tokenizer.json')

In [30]:
# Tối ưu hóa mô hình
#Fine-tune mô hình DeepSeek
#Tải mô hình và tokenizer:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from peft import get_peft_model, LoraConfig

model = AutoModelForCausalLM.from_pretrained("unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit")
tokenizer = AutoTokenizer.from_pretrained("unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit")

# Định nghĩa prompt style
prompt_style = """Below is an instruction that describes a task...
[your prompt template here]
"""

# Tải dataset
dataset = load_dataset("nguyenkimson0202/my-distiset-7cc3ecde", split="train")

# Định nghĩa hàm formatting
def formatting_prompts_func(examples):
    texts = examples["text"]
    formatted_texts = []
    for text in texts:
        formatted_text = prompt_style.format(text, "") + tokenizer.eos_token
        formatted_texts.append(formatted_text)
    return {
        "text": formatted_texts,
    }

# Áp dụng formatting
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)
def tokenize_function(examples):
    outputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    # Sử dụng `input_ids` như `labels` để mô hình dự đoán từ tiếp theo
    outputs["labels"] = outputs["input_ids"].clone()
    return outputs



tokenized_datasets = formatted_dataset.map(tokenize_function, batched=True)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # False nếu không sử dụng Masked Language Modeling
)

# Cấu hình LoRA
lora_config = LoraConfig(
    r=16,
    bias="none"
)

# Thêm LoRA vào mô hình
model = get_peft_model(model, lora_config)
# Điều chỉnh các tham số huấn luyện
training_args = TrainingArguments(
    output_dir='./results_optimized',
    num_train_epochs=5,  # Tăng số epoch
    per_device_train_batch_size=2,  # Điều chỉnh kích thước batch
    gradient_accumulation_steps=4,  # Điều chỉnh tích lũy gradient
    learning_rate=1e-4,  # Điều chỉnh tỷ lệ học
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_optimized',
    fp16=True
)

# Khởi tạo lại Trainer với tham số mới
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator
)

# Huấn luyện lại mô hình
trainer.train()


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Step,Training Loss


TrainOutput(global_step=10, training_loss=6.010838317871094, metrics={'train_runtime': 21.6668, 'train_samples_per_second': 4.615, 'train_steps_per_second': 0.462, 'total_flos': 322930942672896.0, 'train_loss': 6.010838317871094, 'epoch': 3.4})

In [34]:
import torch

# Triển khai mô hình
# Giả sử bạn đang sử dụng GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Chuẩn bị đầu vào và chuyển lên GPU
new_inputs = ["Write a Python function to calculate factorial of a number."]
inputs = tokenizer(new_inputs, return_tensors="pt", padding=True, truncation=True).to(device)

# Chuyển mô hình lên GPU
model.to(device)

# Dự đoán
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=512,
    num_beams=5,
    early_stopping=True
)

# Giải mã kết quả
predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
print("Predictions:", predictions)


Predictions: ['Write a Python function to calculate factorial of a number. The function should return None if the number is not a positive integer.\n</think>\n\nTo write a Python function that calculates the factorial of a number and returns `None` if the number is not a positive integer, follow these steps:\n\n### Step 1: Understand the Problem\nThe factorial of a non-negative integer \\( n \\) is the product of all positive integers less than or equal to \\( n \\). It is denoted by \\( n! \\). For example:\n- \\( 0! = 1 \\)\n- \\( 1! = 1 \\)\n- \\( 2! = 2 \\times 1 = 2 \\)\n- \\( 3! = 3 \\times 2 \\times 1 = 6 \\)\n- \\( 4! = 4 \\times 3 \\times 2 \\times 1 = 24 \\)\n\nIf the input is not a non-negative integer, the function should return `None`.\n\n### Step 2: Define the Function\nCreate a function named `factorial` that takes a single argument `n`. The function should:\n1. Check if `n` is a non-negative integer.\n2. If `n` is not a non-negative integer, return `None`.\n3. If `n` is

In [31]:
# Lưu mô hình và tokenizer
model.save_pretrained('./final_model')
tokenizer.save_pretrained('./final_model')


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/tokenizer.json')

In [28]:
print(tokenized_datasets.column_names)  # Kiểm tra tên cột
print(tokenized_datasets[0])  # Kiểm tra dữ liệu mẫu


['text', 'label', 'input_ids', 'attention_mask', 'labels']
{'text': 'Below is an instruction that describes a task...\n[your prompt template here]\n<｜end▁of▁sentence｜>', 'label': 0, 'input_ids': [151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 1516

In [15]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [29]:
#chạy inference để đánh giá hiệu suất của mô hình:
question = "How can I get the prime numbers from 0 to 125?"

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=2048
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)



Below is an instruction that describes a task...
[your prompt template here]
Now, I will write a prompt for a task. Please follow the step by step...
First, I will select the task.

Next, I will think about the target audience.

Then, I will outline the key features of the task.

After that, I will outline the steps needed to complete the task.

Then, I will consider the necessary resources.

Then, I will outline the expected outputs.

Finally, I will suggest the target format for the prompt.

I will also mention whether I will include specific examples or not.

Now, let's move on to the task.

[The task is to describe a sequence of steps for a specific task.]

I need to write a prompt for this task.
</think>

Certainly! Here's a step-by-step prompt template for describing a sequence of steps:

---

### **Prompt Template: Describe a Sequence of Steps**

1. **Select the Task**:  
   - "Write a detailed step-by-step guide for solving a specific problem or task."

2. **Target Audience**: 